### Test case LVV-T69: Verify implementation of Object Characterization

Verify that Object catalogs produced by the DRP pipeline include all measurements listed in DMS-REQ-0276: a point-source model fit, a bulge-disk model fit, standard colors, a centroid, adaptive moments, Petrosian and Kron fluxes, surface brightness at multiple apertures, proper motion and parallax, and a variability characterization.

In [23]:
import numpy as np
from lsst.daf.butler import Butler

In [2]:
coll = 'LSSTCam/runs/DRP/20250417_20250921/w_2025_49/DM-53545'
butler = Butler('dp2_prep', collections=coll)

In [3]:
obj_refs = butler.query_datasets('object')

In [7]:
obj_schema = butler.get('object.schema', dataId=obj_refs[0].dataId)

Print all column names to the screen:

In [17]:
print(obj_schema.schema.colnames)

['tract', 'patch', 'z_ra', 'z_dec', 'z_raErr', 'z_decErr', 'z_ra_dec_Cov', 'z_psfFlux', 'z_psfFluxErr', 'z_free_psfFlux', 'z_free_psfFluxErr', 'z_free_psfFlux_flag', 'z_bdE1', 'z_bdE2', 'z_bdReB', 'z_bdReD', 'z_bdChi2', 'z_bdFluxB', 'z_bdFluxBErr', 'z_bdFluxD', 'z_bdFluxDErr', 'z_gaapPsfFlux', 'z_gaapPsfFluxErr', 'z_gaap0p7Flux', 'z_gaap0p7FluxErr', 'z_gaap1p0Flux', 'z_gaap1p0FluxErr', 'z_ixx', 'z_iyy', 'z_ixy', 'z_i_flag', 'z_ixxPSF', 'z_iyyPSF', 'z_ixyPSF', 'z_iPSF_flag', 'z_ixxDebiasedPSF', 'z_iyyDebiasedPSF', 'z_ixyDebiasedPSF', 'z_iDebiasedPSF_flag', 'z_moments_uu', 'z_moments_vv', 'z_moments_uv', 'z_moments_a', 'z_moments_b', 'z_moments_theta', 'z_moments_flag', 'z_moments_psf_uu', 'z_moments_psf_vv', 'z_moments_psf_uv', 'z_moments_psf_flag', 'z_moments_round_uu', 'z_moments_round_vv', 'z_moments_round_uv', 'z_moments_round_flag', 'z_moments_psf_debiased_uu', 'z_moments_psf_debiased_vv', 'z_moments_psf_debiased_uv', 'z_moments_psf_debiased_flag', 'z_kronRad', 'z_kronFlux', 'z_kro

Schemas are at [this link](https://github.com/lsst/sdm_schemas/blob/main/python/lsst/sdm/schemas/lsstcam.yaml) -- select a subset of the columns to load from a single `object` table.

In [84]:
columns_to_load = ['i_psfFlux', 'i_psfFluxErr', 'i_bdE1', 'i_bdE2', 'i_bdReB',
                   'i_bdReD', 'i_bdFluxB', 'i_bdFluxBErr', 'i_bdFluxD',
                   'i_bdFluxDErr', 'i_gaapPsfFlux', 'i_gaapPsfFluxErr', 'i_gaap0p7Flux',
                   'i_gaap0p7FluxErr', 'i_gaap1p0Flux', 'i_gaap1p0FluxErr', 'i_ra',
                   'i_dec', 'i_raErr', 'i_decErr', 'i_ixx', 'i_iyy', 'i_ixy',
                   'i_moments_uu', 'i_moments_vv', 'i_moments_uv',
                   'i_cModelFlux', 'i_cModelFluxErr', 'i_cModelFlux_inner',
                   'i_kronRad', 'i_kronFlux', 'i_kronFluxErr',
                   'sersic_reff_x', 'sersic_reff_y', 'sersic_rho', 'g_sersicFlux',
                   'i_sersicFlux', 'r_sersicFlux', 'sersic_index',
                   'i_ap03Flux', 'i_ap03FluxErr', 'i_ap06Flux', 'i_ap06FluxErr',
                   'i_ap09Flux', 'i_ap09FluxErr', 'i_ap12Flux', 'i_ap12FluxErr',
                   'i_ap17Flux', 'i_ap17FluxErr', 'i_ap25Flux', 'i_ap25FluxErr',
                   'i_ap35Flux', 'i_ap35FluxErr', 'i_ap50Flux', 'i_ap50FluxErr',
                   'i_ap70Flux', 'i_ap70FluxErr']

In [85]:
obj_table = butler.get(obj_refs[1], parameters={'columns': columns_to_load})

In [86]:
obj_table[:5]

i_psfFlux,i_psfFluxErr,i_bdE1,i_bdE2,i_bdReB,i_bdReD,i_bdFluxB,i_bdFluxBErr,i_bdFluxD,i_bdFluxDErr,i_gaapPsfFlux,i_gaapPsfFluxErr,i_gaap0p7Flux,i_gaap0p7FluxErr,i_gaap1p0Flux,i_gaap1p0FluxErr,i_ra,i_dec,i_raErr,i_decErr,i_ixx,i_iyy,i_ixy,i_moments_uu,i_moments_vv,i_moments_uv,i_cModelFlux,i_cModelFluxErr,i_cModelFlux_inner,i_kronRad,i_kronFlux,i_kronFluxErr,sersic_reff_x,sersic_reff_y,sersic_rho,g_sersicFlux,i_sersicFlux,r_sersicFlux,sersic_index,i_ap03Flux,i_ap03FluxErr,i_ap06Flux,i_ap06FluxErr,i_ap09Flux,i_ap09FluxErr,i_ap12Flux,i_ap12FluxErr,i_ap17Flux,i_ap17FluxErr,i_ap25Flux,i_ap25FluxErr,i_ap35Flux,i_ap35FluxErr,i_ap50Flux,i_ap50FluxErr,i_ap70Flux,i_ap70FluxErr
float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
3683.386,27.448212,--,--,--,--,--,--,--,--,3529.8147,28.757105,3265.2546,26.938974,3613.4995,31.057043,184.4213017593858,5.9506514910396495,1.7643355e-06,1.7807174e-06,6.7575946,6.813012,-0.13825989,0.2701961,0.2723754,0.0055726734,--,--,--,4.4452524,3944.4246,55.543335,0.49322003,0.5428713,-0.34113595,433.72876,3901.3784,3286.0962,1.3502443,1692.9026,14.574602,3165.869,28.914907,3673.939,43.38716,3927.162,57.88142,4091.1094,82.51837,4229.048,120.92525,4652.71,168.95227,5735.2017,241.22354,8428.688,337.50998
224.58028,26.694267,-1.276938e-05,0.0001537882,0.035381958,0.09374231,223.41516,26.823046,223.8724,26.878424,202.6318,27.727228,196.16019,25.95813,193.22928,29.973951,184.4258825387775,5.950730350421503,2.3902794e-05,2.2505068e-05,5.6244383,7.1766424,-2.7855158,0.22517106,0.2866316,0.11149036,223.49757,26.832943,191.42892,3.2275307,214.46953,39.20218,0.85677516,0.607502,-0.94880044,93.67917,238.23947,189.99124,1.0,109.15272,14.195408,185.44157,28.39404,164.6631,42.726955,126.381454,57.046936,226.23442,81.48908,414.79474,119.684456,794.4218,167.74915,1837.5422,240.2281,3717.7249,337.55017
396.64484,26.371058,-0.122261174,0.41507277,1.8519906,1.8967236,535.895,35.45025,495.83835,32.553177,401.93442,27.722431,356.35104,25.989393,420.3187,29.932459,184.47646533839352,5.9515090714711905,1.5357471e-05,1.9285359e-05,8.787025,10.216259,3.017825,0.35096145,0.40878943,-0.1205072,494.97534,32.496517,452.3924,5.101962,465.38852,62.28068,0.9805892,1.4682097,0.6614468,140.58128,475.722,197.27815,1.0,169.8901,13.983282,391.96426,28.053812,421.67334,42.29053,466.11227,56.50614,469.37015,80.79921,578.3459,118.69968,1075.1515,166.16304,2214.054,237.20978,5281.691,332.37216
559.5891,26.664995,-4.451633e-05,-0.0005800979,0.5968333,1.0293869,586.20667,28.421349,593.4914,28.752779,539.1201,28.11027,498.96353,26.29645,552.4833,30.39486,184.40982243254948,5.952538093156256,1.2095291e-05,1.2499168e-05,7.075966,7.6312337,-0.61345166,0.28297725,0.30504146,0.024600174,592.55145,28.707243,528.49384,3.5651515,581.7439,43.508316,1.134636,0.87191516,-0.5290898,93.20284,632.8145,298.1649,1.0,254.25812,14.142913,492.3221,28.396093,569.66833,42.864464,564.9696,57.35393,579.8277,82.01052,527.7909,120.42194,1302.6085,168.87549,2623.2988,241.5738,5021.0894,338.4564
176.28699,26.319172,-4.6702433e-05,0.00030715094,0.39854532,0.7242797,185.90854,27.169153,187.5445,27.399292,181.2007,27.406609,172.26797,25.665825,186.32384,29.54891,184.44297940962346,5.952339464737323,3.987529e-05,2.8291748e-05,15.2985735,8.060993,8.313343,0.61074954,0.3231889,-0.33269337,188.80553,28.223848,172.0964,7.531584,352.31805,91.33666,0.70070165,1.0809838,0.69777185,218.71178,200.40477,205.00084,1.0,86.21162,13.915051,152.53658,27.911018,180.25536,42.073177,224.94153,56.260544,255.24146,80.58116,502.2244,118.412285,1318.9069,165.766,2867.8994,236.71048,5952.707,331.3748


For each column, display a median and standard deviation to demonstrate that the columns are well-formed and have reasonable values.

In [87]:
for colname in obj_table.colnames:
    if 'Masked' in str(type(obj_table[colname])):
        print(f"{colname} -- median: {np.ma.median(obj_table[colname]):.3}, stdev: {obj_table[colname].std():.3}")
    else:
        print(f"{colname} -- median: {np.nanmedian(obj_table[colname]):.3}, stdev: {np.nanstd(obj_table[colname]):.3}")

i_psfFlux -- median: 3.97e+02, stdev: 8.55e+04
i_psfFluxErr -- median: 54.2, stdev: 36.9
i_bdE1 -- median: -8.14e-06, stdev: 0.182
i_bdE2 -- median: 3.45e-07, stdev: 0.194
i_bdReB -- median: 1.72, stdev: 6.6
i_bdReD -- median: 1.78, stdev: 4.01
i_bdFluxB -- median: 4.87e+02, stdev: 7.06e+04
i_bdFluxBErr -- median: 71.2, stdev: 7.43e+26
i_bdFluxD -- median: 4.73e+02, stdev: 5.09e+04
i_bdFluxDErr -- median: 68.3, stdev: 6.24e+26
i_gaapPsfFlux -- median: 3.96e+02, stdev: 8.52e+04
i_gaapPsfFluxErr -- median: 57.7, stdev: nan
i_gaap0p7Flux -- median: 3.45e+02, stdev: nan
i_gaap0p7FluxErr -- median: 50.7, stdev: nan
i_gaap1p0Flux -- median: 4.1e+02, stdev: 8.87e+04
i_gaap1p0FluxErr -- median: 61.5, stdev: nan
i_ra -- median: 1.84e+02, stdev: 0.38
i_dec -- median: 6.53, stdev: 0.394
i_raErr -- median: 3.24e-05, stdev: 1.61
i_decErr -- median: 3.33e-05, stdev: 4.05
i_ixx -- median: 9.91, stdev: 21.1
i_iyy -- median: 10.0, stdev: 22.2
i_ixy -- median: -0.232, stdev: 10.2
i_moments_uu -- median:

/tmp/ipykernel_4062/3193418656.py:3: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  print(f"{colname} -- median: {np.ma.median(obj_table[colname]):.3}, stdev: {obj_table[colname].std():.3}")


i_sersicFlux -- median: 5.56e+02, stdev: 1.79e+05
r_sersicFlux -- median: 4.18e+02, stdev: 1.56e+05
sersic_index -- median: 1.0, stdev: 0.825
i_ap03Flux -- median: 1.74e+02, stdev: 3.7e+04
i_ap03FluxErr -- median: 29.0, stdev: nan
i_ap06Flux -- median: 3.53e+02, stdev: 7.79e+04
i_ap06FluxErr -- median: 58.4, stdev: nan
i_ap09Flux -- median: 4.32e+02, stdev: 9.51e+04
i_ap09FluxErr -- median: 87.8, stdev: --
i_ap12Flux -- median: 4.72e+02, stdev: 1.03e+05
i_ap12FluxErr -- median: 1.17e+02, stdev: --
i_ap17Flux -- median: 5.19e+02, stdev: 1.14e+05
i_ap17FluxErr -- median: 1.67e+02, stdev: --
i_ap25Flux -- median: 7.05e+02, stdev: 1.43e+05
i_ap25FluxErr -- median: 2.46e+02, stdev: --
i_ap35Flux -- median: 1.28e+03, stdev: 2.19e+05
i_ap35FluxErr -- median: 3.45e+02, stdev: --
i_ap50Flux -- median: 2.46e+03, stdev: 4.07e+05
i_ap50FluxErr -- median: 4.94e+02, stdev: --
i_ap70Flux -- median: 4.67e+03, stdev: 7.78e+05
i_ap70FluxErr -- median: 6.93e+02, stdev: --


### Results:

The above results demonstrated that the following measurements are present in the `object` table produced by DRP processing:

- point-source model fit (and error): `i_psfFlux`, `i_psfFluxErr`
- bulge-disk model fit: `i_bdE1`, `i_bdE2`, `i_bdReB`, `i_bdReD`, `i_bdFluxB`, `i_bdFluxD`
- standard colors: `i_gaapPsfFlux`, `i_gaap0p7Flux`, `i_gaap1p0Flux` (the "GaaP" fluxes, which are ideal for standardized colors of galaxies; as defined in [Kuijken 2008](https://ui.adsabs.harvard.edu/abs/2008A%26A...482.1053K/abstract))
- a centroid: `i_ra`, `i_dec`
- adaptive moments: `i_ixx`, `i_iyy`, `i_ixy`, or `i_moments_uu`, `i_moments_vv`, `i_moments_uv` for moments in a local gnomonic projection
- cModel and Kron fluxes: `i_cModelFlux`, `i_kronFlux`
- surface brightness at multiple apertures: `i_ap03Flux`, `i_ap06Flux`, ... `i_ap70Flux`; aperture fluxes in 3, 6, 9, 12, 17, 25, 35, 50, and 70-pixel apertures

We note that Petrosian fluxes are specified in the requirement, but the project has deemed these unnecessary given the many more useful galaxy flux measurements (e.g., Sersic) we provide.

Missing from the required list are proper motion and parallax, and variability characterization. These will be provided with Data Release 1.

Finally, we note that schemas with more detailed descriptions of the columns can be found at [this link](https://github.com/lsst/sdm_schemas/blob/main/python/lsst/sdm/schemas/lsstcam.yaml), or at the "schema browser" at https://sdm-schemas.lsst.io/.